# Word2Vec

Векторные модели, которые мы рассматривали до этого, условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook).

$$\hat{P}(w_1^T) = \prod_{t=1}^T \hat{P}(w_t \mid w_1^{t-1})$$

Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.

### Зачем это нужно?

* Решать лингвистические задачи (в основном это про семантику и сочетаемость)
* Подавать на вход нейронным сетям

### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W′$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW). 

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


<img src="https://nycdatascience.com/blog/wp-content/uploads/2017/06/cossim.png" width="500">


С помощью дистрибутивных векторных моделей можно строить семантические пропорции и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
1. Матрицы слишком разреженные (→ очень большие)
2. Невозможно установить тип семантических отношений между словами: и синонимы, и антонимы будут одинаково близки, т.к. обычно употребляются в схожих контекстах.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».

<img src="./img/rusvectores2.png" width="500">

Для других языков также можно найти предобученные модели — например, [модели fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (об этих моделях см. ниже).

## Gensim

Использовать предобученную модель w2v или обучить свою можно с помощью библиотеки `gensim`. 

### Как использовать готовую модель

Скачаем с Rusvectores модель для русского языка, обученную на НКРЯ образца 2015 г. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте.

In [1]:
import re
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
from nltk.corpus import stopwords
from gensim.models import word2vec

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

In [ ]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

In [ ]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [ ]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

Находим косинусную близость пары слов:

In [ ]:
print(model.similarity('человек_S', 'обезьяна_S'))

Реши пропорцию!

In [ ]:
print(model.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle). И те, и другие нужны, потому что чем больше данных, тем лучше будет векторная модель!

In [2]:
train = pd.read_csv("./data/w2v/train/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("./data/w2v/train/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("./data/w2v/train/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" \
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов.

In [3]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_wordlist(review, remove_stopwords=False ):
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set...")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...
Parsing sentences from unlabeled set...


D:\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
D:\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.. .'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [6]:
print(len(sentences))
print(sentences[0])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


Обучаем и сохраняем модель word2vec c параметрами, рекомендованными для этого датасета на Kaggle (размерность векторов = 300, минимальная частота слова = 40, размер контекстного окна = 10). Замеряем время обучения модели. 


Праметры:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [10]:
print("Training model...")

%time model = word2vec.Word2Vec(sentences, workers= 4, size= 300, min_count = 40, window = 10, sample = 1e-3)

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Сделаем это и сохраним модель.

In [ ]:
model.init_sims(replace=True)
model_path = "./data/w2v/movie_reviews/movie_reviews.bin"

print("Saving model...")
model.save(model_path)

Смотрим, сколько в модели слов.

In [ ]:
print(len(model.wv.vocab))

Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [ ]:
print(model.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model.most_similar("usa", topn=3))

print(model.doesnt_match("comedy thriller western novel".split()))

## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния о слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификатор мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [ ]:
model.accuracy('./data/w2v/evaluation/ru_analogy_tagged.txt', restrict_vocab=3000000)

### Vecto

Также для "внутренней" (intrinsic) оценки (т.е. в отрыве от конкретной задачи) можно использовать фреймворк [vecto](https://vecto.readthedocs.io/en/latest/).

Установка: `pip3 install git+https://github.com/vecto-ai/vecto.git)`

In [ ]:
from vecto.benchmarks.analogy import Analogy
from vecto.benchmarks.similarity import Similarity
import vecto.embeddings

Vecto использует свои собственные структуры для работы с векторными моделями, поэтому нужно загрузить модель в память ещё раз. Для простоты загрузим только одну из рассмотренных моделей, Word2Vec.

In [ ]:
embeddings = vecto.embeddings.load_from_dir("./data/w2v/movie_reviews/movie_reviews.bin")
embeddings.cache_normalized_copy()

В Vecto можно также как и в Gensim получать вектора для слов или искать наиболее похожие слова из словаря модели:

In [ ]:
embeddings.get_most_similar_words('movie')

In [ ]:
similarity = Similarity()
similarity.get_result(embeddings, path_dataset='./data/w2v/evaluation/similarity'

In [ ]:
analogy = Analogy()
result = analogy.get_result(embeddings, path_dataset='./data/w2v/evaluation/analogy')

## Что еще бывает?

## doc2vec

word2vec с дополнительной меткой id документа, также реализован в `gensim`: `gensim.models.doc2vec`.


![img](img/w2v_4.png)

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грамов. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

https://github.com/facebookresearch/fasttext


## BPE

Модель, основанная на кодировании BPE (Byte Pair Encoding). Это один из способов представления текста, в котором мы используем в качестве токена не слова или символы, а наборы символов в зависимости от глубины кодирования.

Скажем, мы хотим закодировать aaabdaaabac. В этой последовательности сочетание "aa" встречается наиболее часто. Мы можем "слить" эти буквы вместе и рассматривать их как отдельный символ. И так далее итеративно. 

# Задание

Скачайте [данные](https://www.dropbox.com/sh/aa9i9i6yf4tqygz/AABdDpec1N7V9_o9KUSMMpZ0a?dl=0) (корпус новостей на русском языке) и обучите на нем модель word2vec. Можете использовать как оригинальные тексты, так и лемматизированный корпус. 

1. Найдите по 5 ближайших слов к словам "город", "спорт", "бизнес", "Россия", "происшествие", "река", "озеро", "море", "горы", "газпром".
2. Посчитайте семантическую близкость слов "театр" и "кино", "Владивосток" и "Москва", "церковь" и "государство", "культура" и "отдых", "преступление" и "наказание".
3. Решите примеры:
        * москва + екатеринбург - собянин
        * спартак - москва + санкт-петербург
        * иркутск - байкал + сочи
        * татарстан - татарский + бурятия
        * чай - лимон + кофе
        * авиакомпания - аэрофлот + ржд
4. Найдите лишнее, попробуйте интерпретировать результаты
        * магазин, супермаркет, рынок, тц
        * теннис, хоккей, футбол, дзюдо
        * кошка, собака, попугай, кролик
        * коми, дагестан, башкирия, камчатка

Сохраните модель, она вам еще понадобится!